#Import Statements

In [ ]:
#import statements
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains

import json
import time

import pandas as pd

from multiprocessing import Pool

import os


#Create List of URLs

In [ ]:
#function to get listings from URL
from re import U


def get_listings(search_page):
    answer = requests.get(search_page, timeout=5)
    content = answer.content
    soup = BeautifulSoup(content, 'html.parser')

    #get all tags with link
    listings = soup.findAll("a", {"class": "rfexzly dir dir-ltr"})

    url_list=[]

    #loop through URLs and append to list
    for listing in listings:
        href = listing.get('href')
        url_list.append("https://www.airbnb.com"+href)

    #remove duplicate URLs
    urls_final = set(url_list)

    return urls_final

#Input Search URL

In [ ]:
#define search URL
#any weekend in October
airbnb_url = 'https://www.airbnb.com/s/Louisville--Kentucky--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJCROGK48KaYgR_WSJcx2vTro&date_picker_type=flexible_dates&source=structured_search_input_header&search_type=filter_change&ne_lat=38.36561789617469&ne_lng=-85.41682148207042&sw_lat=38.03430121066999&sw_lng=-85.89472675550792&zoom=11&search_by_map=true&flexible_trip_lengths%5B%5D=weekend_trip&flexible_trip_dates%5B%5D=october&query=Louisville%2C%20Kentucky%2C%20United%20States&price_filter_input_type=0&price_min=351'

#AirBnB only allows 300 listings in result

#1st URL - $10 to $100
#2nd URL - $101 to $130
#3rd URL - $131 to $160
#4th URL - $161 to $210
#5th URL - $211 to $250
#6th URL - $251 to $350
#7th URL - $351 to $1000+

In [ ]:
#get listings
listings = get_listings(airbnb_url)

#count listing
listings_count = len(listings)
print("Total listings: "+str(listings_count))

for l in listings:
    print(l)


#Determine the max number of pages for given URL

In [ ]:
import re

answer = requests.get(airbnb_url, timeout=5)
content = answer.content
soup = BeautifulSoup(content, 'html.parser')

num_list = []

#get page numbers
page_numbers = soup.findAll("a", {"class": "_833p2h"})

for pn in page_numbers:
    num = pn.getText()
    num_list.append(int(num))
    

max_page = max(num_list)
print(max_page)
    

#Get URLs from each page

In [ ]:
#function to build list of URLs for each page
def build_urls(main_url, max_page, listings_per_page=20):
    url_list = []
    for i in range(max_page):
        offset = listings_per_page * i
        url_pagination = main_url + f'&items_offset={offset}'
        url_list.append(url_pagination)
    
    return url_list

In [ ]:
url_list = build_urls(airbnb_url, max_page)

url_list

#Create one combined list of URLs to sort through

In [ ]:
listing_count=0
master_list = []

#function to combine all URLs together
def get_all_pages(url_list_all):
    for l in url_list_all:
        master_list.append(get_listings(l))

        global listing_count

        #count listings
        listing_count = len(listings) + listing_count
        print("Total listings: "+str(listing_count))

In [ ]:
get_all_pages(url_list)

#print(master_list)

count=0

for l in master_list:
    count += len(l)

print(count)

#Extract all the details from each listing page

In [ ]:
#from ast import parse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

df = pd.DataFrame([])

#function to get features from each URL
def extract_features(url):

    global df

    title = "Error"
    price = "Error"
    gbb = "Error"
    reviews = "Error"
    parse_coord = "Error"
    days_free = "Error"
    days_booked = "Error"

    options = Options()
    #options.add_argument('--blink-settings=imagesEnabled=false') #need images for Google coordinates
    options.headless = True
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get(url)

    #time.sleep(2)

    try:
        #get title
        title = WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#site-content > div > div:nth-child(1) > div:nth-child(3) > div > div._16e70jgn > div > div:nth-child(1) > div > div > section > div > div > div > div._tqmy57 > div > h2")))
        title = title.text
        #print(title)
        
        #get price
        price = WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#site-content > div > div:nth-child(1) > div:nth-child(3) > div > div._1s21a6e2 > div > div > div:nth-child(1) > div > div > div > div > div > div > div._wgmchy > div._c7v1se > div:nth-child(1) > div > span > div > span._tyxjp1")))
        price = price.text
        #print(price)
        
        #get guest count, bedrooms, bathrooms
        gbb = WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#site-content > div > div:nth-child(1) > div:nth-child(3) > div > div._16e70jgn > div > div:nth-child(1) > div > div > section > div > div > div > div._tqmy57 > ol")))
        gbb = gbb.text
        #print(gbb)

        days_list = []
        days_booked = 0
        days_free = 0

        driver.execute_script("window.scrollTo(0,2000);")

        #get number of days booked and free
        for row in WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "_cvkwaj"))):
            for r in row.find_elements(By.TAG_NAME, 'tr'):
                for cell in r.find_elements(By.TAG_NAME, 'td'):
                    
                    outerHTML = cell.get_attribute("outerHTML")
                    #print(outerHTML)
                    pattern = r'(data-is-day-blocked="true")'

                    try:  
                        is_day_booked = bool(re.search(pattern, outerHTML).group(1))
                    except AttributeError:
                        is_day_booked = "False"

                    if cell.text.isdigit() and is_day_booked == True:
                        days_list.append("Day "+cell.text+" - "+str(is_day_booked))
                        days_booked += 1
                    elif cell.text.isdigit() and is_day_booked == "False":
                        days_list.append("Day "+cell.text+" - "+str(is_day_booked))
                        days_free += 1
                    else: 
                        cell_new = re.sub("[^0-9]", "", cell.text)
                        if cell_new.isdigit() and is_day_booked == True:
                            days_list.append("Day "+cell_new+" - "+str(is_day_booked))
                            days_booked += 1
                        elif cell_new.isdigit() and is_day_booked == "False":
                            days_list.append("Day "+cell_new+" - "+str(is_day_booked))
                            days_free += 1


        driver.execute_script("window.scrollTo(0,3500);")
        
        #get coordinates from Google maps
        coordinates = WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.XPATH, "//a[contains(@href,'https://maps.google.com/maps?ll=')]")))
        coord_href = str(coordinates.get_attribute('href'))

        parse_coord = re.findall(r"ll=(-?\d+\.\d+),(-?\d+\.\d+)", coord_href)

        parse_coord = parse_coord[0]
        # print(parse_coord)

        driver.execute_script("window.scrollTo(0,2200);")

        #get reviews
        reviews = WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#site-content > div > div:nth-child(1) > div:nth-child(4) > div > div > div > div:nth-child(2) > section > div.h1v4rb5q.dir.dir-ltr > span.t1xdm4l6.dir.dir-ltr > h2 > div > span")))
        if type(reviews) != str:
            reviews = reviews.text
        #print(reviews)

    except NoSuchElementException: 
        pass

    except StaleElementReferenceException:
        pass

    except TimeoutException:
        reviews = "No Reviews Yet"
        pass
   

    # print(days_list)
    # print("Days Booked: "+str(days_booked))
    # print("Days Free: "+str(days_free))

    #add variables to DataFrame
    new_row = pd.DataFrame([[title,price,gbb,reviews,str(parse_coord),days_free,days_booked,url]],columns=['title','price','gbb','reviews','coordinates','days_free','days_booked','url'])

    #append row to DataFrame
    df = df.append(new_row)

    #display(df)

    driver.quit()

#Insert Dataframe to MySQL

In [ ]:
#insert DataFrame rows to MySQL
import pandas as pd
from sqlalchemy import create_engine
import pymysql
from urllib import parse


engine = create_engine(
  'mysql+pymysql://{user}:{password}@{account}'.format(
    #removed creds
    user='',
    password=parse.quote(''),
    account='',
  ) 
)

#time to extract some features

for urls in master_list:
    for page in urls:
        print(page)
        print(len(df.index))

        extract_features(page)


df.to_sql("airbnb_listings", if_exists='append', index=False, con=engine)


#Reverse Geocode coordinates to get Zip Code with geopy

In [ ]:
import geopy
import pandas as pd
import csv 
from geopy.point import Point


geolocator = geopy.Nominatim(user_agent="airbnb")

#removed file path
with open('/') as file:
    reader = csv.DictReader(file)
    for row in reader:
        record = []
        lat=row['latitude']
        lon=row['longitude']
        url=row['url']
        location = geolocator.reverse(Point(lat,lon))
        print(location)

        try:
            zip = location.raw['address']['postcode']
        except KeyError:
            pass
        record.append(lat)
        record.append(lon)
        record.append(url)
        record.append(zip)
        print(record)

        #removed file path
        with open('', 'a', newline='', encoding='utf-8') as op_file:
            wr = csv.writer(op_file)
            wr.writerow(record)
